In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/research/

/content/drive/MyDrive/research


In [3]:
%%capture
! pip install datasets transformers

In [ ]:
! pip freeze


In [ ]:
from google.colab import files
files.upload()

In [4]:
import json

# New Section

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ghosh-r/bangla-gpt2")

train_path = 'train.txt'
test_path = 'test.txt'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [20]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("ghosh-r/bangla-gpt2")



training_args = TrainingArguments(
    output_dir="./robi-kobita",
    overwrite_output_dir=True,
    num_train_epochs=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    eval_steps = 80,
    save_steps=500, 
    warmup_steps=100,
    logging_steps=250,
    prediction_loss_only=True
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1295: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [35]:
training_args = TrainingArguments(
    output_dir="./robi-kobita",
    overwrite_output_dir=True,
    num_train_epochs=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    eval_steps = 80,
    save_steps=500, 
    warmup_steps=100,
    logging_steps=250,
    prediction_loss_only=True
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [36]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50266, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-6): 7 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50266, bias=False)
)

In [38]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig

config = AutoConfig.from_pretrained(
    "ghosh-r/bangla-gpt2",
    #"gpt2",
    vocab_size=len(tokenizer),
   
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    num_hidden_layers=7
)
model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"Model size: {model_size/1000**2:.1f} M parameters")


Model size: 89.0 M parameters


In [39]:
import matplotlib.pyplot as plt
plt.figure(1)
plt.plot(global_step)
plt.plot(train_loss)
plt.title('Step vs Loss')
plt.ylabel('Training Loss')
plt.xlabel('Step')
plt.legend(['step', 'loss'], loc='upper left')

NameError: ignored

<Figure size 640x480 with 0 Axes>

In [37]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
250,5.227200
500,2.864500
750,1.620100
1000,0.682400
1250,0.256400
1500,0.117900
1750,0.077300
2000,0.060900
2250,0.050700
2500,0.044000


TrainOutput(global_step=4500, training_loss=0.6238560072580973, metrics={'train_runtime': 1821.1392, 'train_samples_per_second': 39.261, 'train_steps_per_second': 2.471, 'total_flos': 2724548935680000.0, 'train_loss': 0.6238560072580973, 'epoch': 100.0})

In [45]:
trainer.evaluate()


{'eval_loss': 6.268988609313965,
 'eval_runtime': 0.1748,
 'eval_samples_per_second': 91.557,
 'eval_steps_per_second': 5.722,
 'epoch': 100.0}

In [46]:
trainer.save_model()

In [47]:
tokenizer.save_pretrained('./tokenizer')

('./tokenizer/tokenizer_config.json',
 './tokenizer/special_tokens_map.json',
 './tokenizer/vocab.json',
 './tokenizer/merges.txt',
 './tokenizer/added_tokens.json',
 './tokenizer/tokenizer.json')

In [48]:
from transformers import pipeline

poet = pipeline('text-generation', model='ghosh-r/robi-kobi', tokenizer='ghosh-r/bangla-gpt2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [66]:
poem = poet('ওরে ছাগল, বল্‌ত আগে সুড় সুড়িটা কেমন লাগে?', max_length=300)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [67]:
poem[0]['generated_text'].replace('\n', ' ').replace('\u200c', '').replace('\xa0', '')

'ওরে ছাগল, বল্ত আগে সুড় সুড়িটা কেমন লাগে? বল্নু ভাই কেমন রাজদণ্ড দেখায় বুঝিবা শইল বলতে তো দিতে চাহি নাকো আর, আমি নিয়ো যে কাজটা, কাজটা পাঠাই দিলে বলে পাঠাই সবে বল্নু কি দোষে লাগে সে আমার? বল্তে তো দিতে না পারি যে আমি শুধু বল্নু র’বে মুখে কি বল্তে, বল্তে না পারি যে সিঞ্চিত অধরে বল্তে না পারি পাঁজরটা গিঁঠ বাঁধা কেন যে? বল্নু এ মুখ কখনো দেখিবি কি হাওয়া? বল্নু শু গেঁা গিরে দে গায়ারে পারেখ্য়ে সব জাজিলায়া নারেনু বল্ ক'

In [51]:
string = str(poem[0]['generated_text'].replace('\n', ' ').replace('\xa0', '')).replace(', ', ',\n')

আমি তোমারে দেখেছি মোর হিয়া মাঝে সূচিত্রা গৃহদাহে।শুভ তোমারি সাধন-মরণ শূন্যে আমারি মুখশিখায় বেড়ে উঠার স্বপ্নে। তুমি আমারি হেরি মোর শির তুমি তোমারি প্রাণের মন্ত্র, হেরি আমারি অন্যমনে! মোরে খুঁজেছি তোমার নিত্য ভাই তুমি এসেছ আমার চেয়ে। এ তো জীবনের মন্ত্র হয়েছে আপন